In [8]:
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [9]:
import os

def get_file_class_map(root_dir):
    file_class_map = {}

    for root, dirs, files in os.walk(root_dir):
        class_name = os.path.basename(root)

        if class_name == os.path.basename(root_dir):
            continue

        for file_name in files:
            key = file_name.split('.')[0]
            file_class_map[key] = class_name

    return file_class_map

test_directory = '../../dataset/20news-bydate-test'
file_class_test_map = get_file_class_map(test_directory)

train_directory = '../../dataset/20news-bydate-train'
file_class_train_map = get_file_class_map(train_directory)

file_class_map = file_class_test_map | file_class_train_map

print(file_class_map['53068'])
print(file_class_map['38761'])
print(file_class_map['49960'])

alt.atheism
comp.graphics
alt.atheism


In [10]:
def load_embeddings(file_path):
    embeddings = []
    doc_ids = []
    with open(file_path, 'r') as f:
        for line in f:
            data = line.strip().split('\t')
            doc_id = data[0]
            embedding = list(map(float, data[1:]))
            doc_ids.append(doc_id)
            embeddings.append(embedding)
    return np.array(embeddings), doc_ids

def get_labels(doc_ids, file_class_map):
    labels = [file_class_map[doc_id] for doc_id in doc_ids]
    return labels

In [11]:
from sklearn.preprocessing import LabelEncoder

train_embedding_file_path = '../../nechkasova-vectorization/assets/annotated-corpus/train.tsv'
test_embedding_file_path = '../../nechkasova-vectorization/assets/annotated-corpus/test.tsv'

X_train, doc_ids = load_embeddings(train_embedding_file_path)
y_train = get_labels(doc_ids, file_class_map)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

X_test, doc_ids = load_embeddings(test_embedding_file_path)
y_test = get_labels(doc_ids, file_class_map)

label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

In [12]:
print(X_train, X_test, y_train_encoded, y_test_encoded)

[[ 0.20773591  0.72988412 -0.11352285 ... -0.29935595 -0.1885143
   0.286544  ]
 [ 0.14539602  0.61061025  0.04516278 ... -0.24808198 -0.27161203
   0.3643503 ]
 [ 0.18635802  0.51708716 -0.02916354 ... -0.17222796 -0.28976381
   0.04486433]
 ...
 [ 0.19553041  0.5397074  -0.04884807 ... -0.34322487 -0.2403506
  -0.01544757]
 [ 0.26425181  0.69465227 -0.04891185 ... -0.29389333 -0.26407785
   0.18813769]
 [ 0.20166849  0.6093968  -0.02778901 ... -0.29972352 -0.31026409
   0.2870507 ]] [[ 0.08262633  0.39071409 -0.00992401 ... -0.21398073 -0.2028388
   0.24255998]
 [ 0.08318808  0.51211188 -0.16336193 ... -0.20390385 -0.2129409
   0.11551128]
 [ 0.11453234  0.67152243 -0.02109246 ... -0.1015598  -0.25121764
   0.19826109]
 ...
 [ 0.18724539  0.53250695  0.00307958 ... -0.22663249 -0.2919248
   0.29681634]
 [ 0.27671094  0.50519299  0.01900373 ... -0.30224663 -0.279837
   0.16828765]
 [ 0.04503434  0.5982903  -0.10858543 ... -0.21932402 -0.2808188
   0.16079474]] [ 6 17  6 ... 19 19 19] 

In [13]:
def calculate_metrics(y_true, y_pred):
    labels = np.unique(y_true)
    precision_per_class = []
    recall_per_class = []
    f1_score_per_class = []
    
    total_samples = len(y_true)
    
    for label in labels:
        tp = sum((y_true == label) & (y_pred == label))
        fp = sum((y_true != label) & (y_pred == label))
        fn = sum((y_true == label) & (y_pred != label))
        tn = sum((y_true != label) & (y_pred != label))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        precision_per_class.append(precision)
        recall_per_class.append(recall)
        f1_score_per_class.append(f1_score)
    
    accuracy = np.sum(y_true == y_pred) / total_samples
    
    return {
        'precision': np.mean(precision_per_class),
        'recall': np.mean(recall_per_class),
        'f1-score': np.mean(f1_score_per_class),
        'accuracy': accuracy
    }

In [14]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
kernel_params = {
    'linear': {'C': [0.1, 1, 10]},
    'poly': {'C': [0.1, 1], 'degree': [2, 3], 'coef0': [0.1, 1]},
    'rbf': {'C': [0.1, 1], 'gamma': ['scale', 0.1]},
    'sigmoid': {'C': [0.1, 1], 'gamma': ['scale', 0.1], 'coef0': [0, 0.5]}
}

In [15]:
def experiment_svm(X_train, y_train, X_test, y_test, kernels, kernel_params):
    results = {}

    for kernel in kernels:
        param_values = kernel_params[kernel]
        
        for C in param_values.get('C', [1]):
            for degree in param_values.get('degree', [3]):
                for gamma in param_values.get('gamma', ['scale']):
                    for coef0 in param_values.get('coef0', [0]):
                        model_params = {'kernel': kernel, 'C': C}
                        if kernel == 'poly':
                            model_params['degree'] = degree
                            model_params['coef0'] = coef0
                        elif kernel in ['rbf', 'sigmoid']:
                            model_params['gamma'] = gamma
                            model_params['coef0'] = coef0

                        model = SVC(**model_params, random_state=42)
                        start_time = time.time()
                        model.fit(X_train, y_train)
                        train_time = time.time() - start_time

                        y_pred = model.predict(X_test)

                        metrics = calculate_metrics(y_test, y_pred)

                        results[(kernel, C, degree, gamma, coef0)] = {
                            'metrics': metrics,
                            'training_time': train_time
                        }

                        print(f"Kernel: {kernel}, C: {C}, Degree: {degree}, Gamma: {gamma}, Coef0: {coef0}")
                        print(f"Accuracy: {metrics['accuracy']:.4f}, Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, F1-Score: {metrics['f1-score']:.4f}")
                        print(f"Training Time: {train_time:.4f} seconds\n")
    return results

In [16]:
# SVM (линейное ядро)
results = experiment_svm(X_train, y_train_encoded, X_test, y_test_encoded, kernels, kernel_params)

print("SVM (линейное ядро) - метрики")
for params, metrics in results.items():
    print(f"Параметры: {params}, Метрики: {metrics}")

Kernel: linear, C: 0.1, Degree: 3, Gamma: scale, Coef0: 0
Accuracy: 0.2387, Precision: 0.2066, Recall: 0.2075, F1-Score: 0.1723
Training Time: 6.2211 seconds

Kernel: linear, C: 1, Degree: 3, Gamma: scale, Coef0: 0
Accuracy: 0.2788, Precision: 0.2597, Recall: 0.2475, F1-Score: 0.2235
Training Time: 5.4420 seconds

Kernel: linear, C: 10, Degree: 3, Gamma: scale, Coef0: 0
Accuracy: 0.3139, Precision: 0.3063, Recall: 0.2829, F1-Score: 0.2636
Training Time: 6.9223 seconds

Kernel: poly, C: 0.1, Degree: 2, Gamma: scale, Coef0: 0.1
Accuracy: 0.1782, Precision: 0.1875, Recall: 0.1479, F1-Score: 0.1211
Training Time: 7.3157 seconds

Kernel: poly, C: 0.1, Degree: 2, Gamma: scale, Coef0: 1
Accuracy: 0.2063, Precision: 0.1485, Recall: 0.1732, F1-Score: 0.1366
Training Time: 6.8237 seconds

Kernel: poly, C: 0.1, Degree: 3, Gamma: scale, Coef0: 0.1
Accuracy: 0.1831, Precision: 0.2144, Recall: 0.1514, F1-Score: 0.1239
Training Time: 7.1892 seconds

Kernel: poly, C: 0.1, Degree: 3, Gamma: scale, Coef

Параметры: ('poly', 1, 3, 'scale', 1), Метрики: {'metrics': {'precision': 0.521016335174504, 'recall': 0.363963840727409, 'f1-score': 0.36057352794214004, 'accuracy': 0.4193919038359554}, 'training_time': 5.166396856307983}

In [17]:
hidden_layer_options = [(50,), (100,), (100, 50), (100, 100, 50)]
max_iter_options = [100, 300, 500, 1000]
learning_rate_options = [0.001, 0.01]
activation_options = ['relu', 'tanh', 'logistic']

In [18]:
def experiment_mlp(X_train, y_train, X_test, y_test):
    results = {}
    
    for hidden_layers in hidden_layer_options:
        for max_iter in max_iter_options:
            for learning_rate in learning_rate_options:
                for activation in activation_options:
                    model = MLPClassifier(
                        hidden_layer_sizes=hidden_layers,
                        max_iter=max_iter,
                        learning_rate_init=learning_rate,
                        activation=activation,
                        random_state=42
                    )

                    start_time = time.time()
                    model.fit(X_train, y_train)
                    training_time = time.time() - start_time
                    
                    y_pred = model.predict(X_test)
                    metrics = calculate_metrics(y_test, y_pred)

                    params = (hidden_layers, max_iter, learning_rate, activation)
                    results[params] = {
                        'metrics': metrics,
                        'training_time': training_time
                    }

                    print(f"Params: {params}")
                    print(f"Metrics: {metrics}")
                    print(f"Training Time: {training_time:.4f} seconds\n")
    
    return results

In [19]:
# MLP
results = experiment_mlp(X_train, y_train_encoded, X_test, y_test_encoded)

print("MLP - метрики:")
for params, result in results.items():
    print(f"Params: {params}, Metrics: {result['metrics']}, Training Time: {result['training_time']:.4f} seconds")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Params: ((50,), 100, 0.001, 'relu')
Metrics: {'precision': 0.2887943548147237, 'recall': 0.2835544638488966, 'f1-score': 0.25665263582757414, 'accuracy': 0.31240042485395647}
Training Time: 2.8510 seconds



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Params: ((50,), 100, 0.001, 'tanh')
Metrics: {'precision': 0.2861109979379039, 'recall': 0.28341303677576357, 'f1-score': 0.25402041764228234, 'accuracy': 0.3053637812002124}
Training Time: 2.7025 seconds



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Params: ((100, 100, 50), 500, 0.001, 'tanh')
Metrics: {'precision': 0.2648569640089752, 'recall': 0.27021102591243074, 'f1-score': 0.25204377390008187, 'accuracy': 0.2773499734466277}
Training Time: 68.8698 seconds



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Params: ((100, 100, 50), 500, 0.001, 'logistic')
Metrics: {'precision': 0.27470184843809786, 'recall': 0.287920272803822, 'f1-score': 0.2551199155768756, 'accuracy': 0.3032395114179501}
Training Time: 71.5932 seconds

Params: ((100, 100, 50), 500, 0.01, 'relu')
Metrics: {'precision': 0.2626198375794607, 'recall': 0.2658610687687185, 'f1-score': 0.24739517650905535, 'accuracy': 0.28690918746680827}
Training Time: 22.8926 seconds

Params: ((100, 100, 50), 500, 0.01, 'tanh')
Metrics: {'precision': 0.2579859690595285, 'recall': 0.24741675703736346, 'f1-score': 0.23513458079782312, 'accuracy': 0.26394052044609667}
Training Time: 18.0362 seconds

Params: ((100, 100, 50), 500, 0.01, 'logistic')
Metrics: {'precision': 0.23742889233707967, 'recall': 0.2403316862210297, 'f1-score': 0.2259100875794676, 'accuracy': 0.24787573021773765}
Training Time: 38.1466 seconds

Params: ((100, 100, 50), 1000, 0.001, 'relu')
Metrics: {'precision': 0.2784795613331424, 'recall': 0.281212607573238, 'f1-score': 0.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Params: ((100, 100, 50), 1000, 0.001, 'logistic')
Metrics: {'precision': 0.2844620753807278, 'recall': 0.28914832191050766, 'f1-score': 0.2629028688513816, 'accuracy': 0.30231014338821033}
Training Time: 171.7048 seconds

Params: ((100, 100, 50), 1000, 0.01, 'relu')
Metrics: {'precision': 0.2626198375794607, 'recall': 0.2658610687687185, 'f1-score': 0.24739517650905535, 'accuracy': 0.28690918746680827}
Training Time: 22.9529 seconds

Params: ((100, 100, 50), 1000, 0.01, 'tanh')
Metrics: {'precision': 0.2579859690595285, 'recall': 0.24741675703736346, 'f1-score': 0.23513458079782312, 'accuracy': 0.26394052044609667}
Training Time: 17.2888 seconds

Params: ((100, 100, 50), 1000, 0.01, 'logistic')
Metrics: {'precision': 0.23742889233707967, 'recall': 0.2403316862210297, 'f1-score': 0.2259100875794676, 'accuracy': 0.24787573021773765}
Training Time: 39.2774 seconds

MLP - метрики:
Params: ((50,), 100, 0.001, 'relu'), Metrics: {'precision': 0.2887943548147237, 'recall': 0.2835544638488966, 

Params: ((100, 100, 50), [300, 500, 1000], 0.01, 'logistic'), Metrics: {'precision': 0.9321507597441017, 'recall': 0.9333474888506172, 'f1-score': 0.932394304773428, 'accuracy': 0.9355665547109776}, Training Time: 33.0178 seconds

In [20]:
def safe_log1p(embeddings, shift=True):
    if shift:
        min_value = embeddings.min()
        if min_value < -1:
            embeddings_shifted = embeddings + abs(min_value) + 1
        else:
            embeddings_shifted = embeddings
        return np.log1p(embeddings_shifted)
    else:
        return np.log1p(np.abs(embeddings))

transformations = [
    safe_log1p,
    np.sin,
    np.cos,
    np.square
]

def extend_embeddings(embeddings, functions):
    extended_embeddings = embeddings.copy()
    for func in functions:
        transformed = func(embeddings)
        extended_embeddings = np.concatenate((extended_embeddings, transformed), axis=1)
    return extended_embeddings

In [21]:
X_train_extended = extend_embeddings(X_train, transformations)
X_test_extended = extend_embeddings(X_test, transformations)

In [22]:
def experiment_mlp_extended(X_train, y_train, X_test, y_test):
    model = MLPClassifier(hidden_layer_sizes=(100, 100, 50), max_iter=300, learning_rate_init=0.01, activation='logistic')
    
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    
    metrics = calculate_metrics(y_test, y_pred)
    
    return metrics, training_time

In [23]:
metrics_mlp, training_time_mlp = experiment_mlp_extended(X_train_extended, y_train_encoded, X_test_extended, y_test_encoded)
print(metrics_mlp)
print("Training Time:", training_time_mlp)

{'precision': 0.24003695862342753, 'recall': 0.2399057659308343, 'f1-score': 0.21673998982477238, 'accuracy': 0.26433882103027084}
Training Time: 49.51618528366089


In [24]:
X_train_extended = extend_embeddings(X_train, [np.sin])
X_test_extended = extend_embeddings(X_test, [np.sin])

metrics_mlp, training_time_mlp = experiment_mlp_extended(X_train_extended, y_train_encoded, X_test_extended, y_test_encoded)
print(metrics_mlp)
print("Training Time:", training_time_mlp)

{'precision': 0.24035001203467177, 'recall': 0.24735044200114595, 'f1-score': 0.2268078620400887, 'accuracy': 0.25185873605947956}
Training Time: 42.9942889213562


In [25]:
X_train_extended = extend_embeddings(X_train, [np.square])
X_test_extended = extend_embeddings(X_test, [np.square])

metrics_mlp, training_time_mlp = experiment_mlp_extended(X_train_extended, y_train_encoded, X_test_extended, y_test_encoded)
print(metrics_mlp)
print("Training Time:", training_time_mlp)

{'precision': 0.24301578285239017, 'recall': 0.24410509110262782, 'f1-score': 0.23023554968705393, 'accuracy': 0.2569038767923526}
Training Time: 69.53520083427429


In [26]:
X_train_extended = extend_embeddings(X_train, [safe_log1p])
X_test_extended = extend_embeddings(X_test, [safe_log1p])

metrics_mlp, training_time_mlp = experiment_mlp_extended(X_train_extended, y_train_encoded, X_test_extended, y_test_encoded)
print(metrics_mlp)
print("Training Time:", training_time_mlp)

{'precision': 0.26035749634242544, 'recall': 0.24750608656332745, 'f1-score': 0.22119166861716977, 'accuracy': 0.2741635687732342}
Training Time: 68.3126540184021


In [27]:
X_train_extended = extend_embeddings(X_train, [np.sin, np.square])
X_test_extended = extend_embeddings(X_test, [np.sin, np.square])

metrics_mlp, training_time_mlp = experiment_mlp_extended(X_train_extended, y_train_encoded, X_test_extended, y_test_encoded)
print(metrics_mlp)
print("Training Time:", training_time_mlp)

{'precision': 0.24431156260498935, 'recall': 0.257920322506607, 'f1-score': 0.23254087803275972, 'accuracy': 0.25677110993096125}
Training Time: 67.94425988197327
